## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_2 = pd.read_csv("../data/gcc8/o0123합본/o2/o2_gcc8.csv", index_col=0)
core8_2 = pd.read_csv("../data/gcc8/o0123합본/o2/o2_core8.csv", index_col=0)
print(bin8_2.shape, core8_2.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_2.reset_index(inplace=True, drop=True)
core8_2.reset_index(inplace=True, drop=True)

print('bin - core concat')
bin8_2 = pd.concat([bin8_2, core8_2], axis=0)
print('shape -', bin8_2.shape)
print('reset_index 완료')
print('input data shape')
bin8_2.head()

(21129497, 2) (7615203, 2)
bin - core concat
shape - (28744700, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_2['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_2['label'].value_counts())

256
0    28706035
1       38665
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_2[bin8_2['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_2), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_2', len(ls_idx_bin))

# loc 로 수정필요
bin8_2_Ngram = bin8_2.loc[ls_idx_bin,:].copy()

1237280
나머지 0
최종 길이 1237280
bin8_2 1237280


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_2_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_2.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_2)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_2[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

61202.0
0 11194137
1000 13707423
2000 1204184
3000 15175726
4000 8436997
5000 23867767
6000 17865476
7000 3410575
8000 23738651
9000 15651481
10000 8308557
11000 11094019
12000 9676867
13000 7367917
14000 19047549
15000 5915394
16000 3819799
17000 18123803
18000 24702489
19000 12300999
20000 5020131
21000 13850170
22000 6955185
23000 13690173
24000 9144428
25000 25567588
26000 16182011
27000 14365765
28000 12034638
29000 10089321
30000 18290396
31000 6631020
32000 11324555
33000 3237198
34000 13952662
35000 5157734
36000 24690952
37000 356997
38000 13161408
39000 20861069
40000 21516870
41000 21921554
42000 1421332
43000 8106098
44000 19748373
45000 16384134
46000 25579802
47000 8458456
48000 17090122
49000 25016332
50000 20951940
51000 14052821
52000 5023210
53000 27562185
54000 3492722
55000 18251106
56000 4726712
57000 19593222
58000 3004539
59000 25207396
60000 20790288
61000 23913945
완료
61202


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_2['label'].value_counts()[1])

61202
38665


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_2_Ngram])
final.shape

(3916928, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_2_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_2_onehot_Ngram = pd.concat([final['label'], bc8_2_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_2_onehot_Ngram.shape)

원핫인코딩완료
(3916928, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_2 = bc8_2_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_2 = bc8_2_onehot_Ngram['label'].to_numpy()
print(x_bc8_2.shape, x_bc8_2.shape)

x_bc8_2 = x_bc8_2.reshape(-1, right_idx, x_bc8_2.shape[1])
y_bc8_2 = y_bc8_2.reshape(-1, right_idx, 1)

print(x_bc8_2.shape, y_bc8_2.shape)

(3916928, 256) (3916928, 256)
(122404, 32, 256) (122404, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_2.shape[0])

x_bc8_2 = x_bc8_2[p]
y_bc8_2 = y_bc8_2[p]

print(x_bc8_2.shape, y_bc8_2.shape)

(122404, 32, 256) (122404, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_2, y_bc8_2):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_2[train],
               y_bc8_2[train],
               epochs = 10,
               batch_size = 128,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_2[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_2[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 10s 95us/sample - loss: 0.0264 - accuracy: 0.9929
Epoch 2/10
110163/110163 [==============================] - 6s 58us/sample - loss: 0.0077 - accuracy: 0.9975
Epoch 3/10
110163/110163 [==============================] - 6s 58us/sample - loss: 0.0060 - accuracy: 0.9981
Epoch 

accuracy_score 0.9989124663017728
recall_score 0.9564397905759162
precision_score 0.9544409613375131
f1_score 0.955439330543933
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 9s 81us/sample - loss: 0.0020 - accuracy: 0.9994
Epoch 2/10
110163/110163 [==============================] - 6s 58us/sample - loss: 0.0017 - acc

accuracy_score 0.9993924107507557
recall_score 0.9700949686805416
precision_score 0.9815988550398691
f1_score 0.9758130081300813
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 9s 81us/sample - loss: 7.6496e-04 - accuracy: 0.9998
Epoch 2/10
110163/110163 [==============================] - 6s 58us/sample - loss: 6.6386e

accuracy_score 0.9995940895351687
recall_score 0.9872375463153561
precision_score 0.9801757612916411
f1_score 0.9836939801046046
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110163 samples
Epoch 1/10
110163/110163 [==============================] - 10s 90us/sample - loss: 3.4076e-04 - accuracy: 0.9999
Epoch 2/10
110163/110163 [==============================] - 6s 59us/sample - loss: 2.4990

accuracy_score 0.999765133567519
recall_score 0.9879492145470196
precision_score 0.99221958072185
f1_score 0.9900797929695924
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 9s 81us/sample - loss: 1.6940e-04 - accuracy: 0.9999
Epoch 2/10
110164/110164 [==============================] - 6s 59us/sample - loss: 1.4743e-04

...

accuracy_score 0.9999591503267974
recall_score 0.9991793188346327
precision_score 0.9975419909873003
f1_score 0.998359983599836
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 10s 94us/sample - loss: 6.3473e-05 - accuracy: 1.0000
Epoch 2/10
110164/110164 [==============================] - 6s 59us/sample - loss: 4.8901e

accuracy_score 0.9999821282679738
recall_score 0.9995760915642221
precision_score 0.9989409023511968
f1_score 0.9992583960165272
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 9s 81us/sample - loss: 4.5000e-05 - accuracy: 1.0000
Epoch 2/10
110164/110164 [==============================] - 6s 59us/sample - loss: 2.6959e

accuracy_score 0.9999770220588236
recall_score 0.9997840639170805
precision_score 0.9982751185855973
f1_score 0.9990290214694141
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 9s 81us/sample - loss: 4.5267e-05 - accuracy: 1.0000
Epoch 2/10
110164/110164 [==============================] - 6s 59us/sample - loss: 2.8725e

accuracy_score 0.9999821282679738
recall_score 0.9995773457311918
precision_score 0.9989440337909187
f1_score 0.9992605894158656
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 110164 samples
Epoch 1/10
110164/110164 [==============================] - 9s 82us/sample - loss: 3.0399e-05 - accuracy: 1.0000
Epoch 2/10
110164/110164 [==============================] - 6s 59us/sample - loss: 3.1906e

accuracy_score 0.9999923406862745
recall_score 0.9997948717948718
precision_score 0.9995898277276456
f1_score 0.9996923392472568

K-fold cross validation Accuracy: [0.9989124663017728, 0.9993924107507557, 0.9995940895351687, 0.999765133567519, 0.9999157475490196, 0.9999591503267974, 0.9999821282679738, 0.9999770220588236, 0.9999821282679738, 0.9999923406862745]

K-fold cross validation Recall: [0.9564397905759162, 0.9700949686805416, 0.9872375463153561, 0.9879492145470196, 0.9966237602869804, 0.9991793188346327, 0.9995760915642221, 0.9997840639170805, 0.9995773457311918, 0.9997948717948718]

K-fold cross validation Precision: [0.9544409613375131, 0.9815988550398691, 0.9801757612916411, 0.99221958072185, 0.9964135021097047, 0.9975419909873003, 0.9989409023511968, 0.9982751185855973, 0.9989440337909187, 0.9995898277276456]

K-fold cross validation F1-Score: [0.955439330543933, 0.9758130081300813, 0.9836939801046046, 0.9900797929695924, 0.9965186201076064, 0.998359983599836, 0.99925839601

## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9997472617312079
10-Fold Cross_validation. Recall : 0.9896256972247814
10-Fold Cross_validation. Precision : 0.9898140533943236
10-Fold Cross_validation. F1-Score : 0.9897145061604717


In [13]:
model1.save('gcc8_bin_core_s32_h48_o2.h5')
print('save 완료')

save 완료
